# Computer Vision | Homework 08
### Mohsen Ebadpour | 400131080 | m.ebadpour@aut.ac.ir

In [ ]:
import cv2 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 
import time 
plt.rcParams['figure.figsize'] = [20, 15]

# Part-1: Find parameters for goodFeaturesToTrack

In [ ]:
video_address = "./02.mp4"
video = cv2.VideoCapture(video_address)
ret, current_frame = video.read()
height, width, channels = current_frame.shape
ratio = 0.5
current_frame = cv2.resize(current_frame, (int(width*ratio), int(height*ratio)))
current_frame = cv2.rotate(current_frame, cv2.ROTATE_180)
current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)


def fine_tune(flag=True, current_frame=np.array([]), current_frame_gray=np.array([]),use_harris_detector=False,abslut=[]):
    _max_corners = [1000,500,200,100,50]
    _quality_level = [0.6,0.3,0.15,0.1,0.05]
    _min_distance = [5,10,20,50,100]
    _block_size = [5,15,20,30,50]

    ConstMaxCorners = 1000
    ConstQualityLevel = 0.3
    ConstMinDistance = 5
    ConstBolkSize = 5
    
    if len(abslut) > 0:
        current_frame_color = current_frame.copy()
        features = cv2.goodFeaturesToTrack(current_frame_gray, maxCorners=abslut[0], qualityLevel=abslut[1], minDistance=abslut[2], mask=None, blockSize=abslut[3], useHarrisDetector=use_harris_detector)
        for i in range(len(features)):
            cv2.circle(current_frame_color, (int(features[i][0][0]), int(features[i][0][1])), 2, (255,0,0), 2)
        cv2.imwrite("./P1-all-fine-tune.jpg", current_frame_color)
        return current_frame_color

    if flag:
        ROW_IMGS = []
        COL_IMGS = []
        for quality_level in _quality_level:
            for block_size in _block_size:
                current_frame_color = current_frame.copy()
                features = cv2.goodFeaturesToTrack(current_frame_gray, maxCorners=ConstMaxCorners, qualityLevel=quality_level, minDistance=ConstMinDistance, mask=None, blockSize=block_size, useHarrisDetector=use_harris_detector)
                for i in range(len(features)):
                    cv2.circle(current_frame_color, (int(features[i][0][0]), int(features[i][0][1])), 2, (255,0,0), 2)
                COL_IMGS.append(current_frame_color)
                
            tmp_img = cv2.hconcat(COL_IMGS)
            COL_IMGS = []
            ROW_IMGS.append(tmp_img)
        cv2.imwrite("./P1-fine-tune quality_level and block_size.jpg", cv2.vconcat(ROW_IMGS))
        
    else:
        ROW_IMGS = []
        COL_IMGS = []
        for max_corners in _max_corners:
            for min_distance in _min_distance:
                current_frame_color = current_frame.copy()
                features = cv2.goodFeaturesToTrack(current_frame_gray, maxCorners=max_corners, qualityLevel=ConstQualityLevel, minDistance=min_distance, mask=None, blockSize=ConstBolkSize, useHarrisDetector=use_harris_detector)
                for i in range(len(features)):
                    cv2.circle(current_frame_color, (int(features[i][0][0]), int(features[i][0][1])), 2, (255,0,0), 2)
                COL_IMGS.append(current_frame_color)
                
            tmp_img = cv2.hconcat(COL_IMGS)
            COL_IMGS = []
            ROW_IMGS.append(tmp_img)

        cv2.imwrite("./P1-fine-tune max_corners and min_distance.jpg", cv2.vconcat(ROW_IMGS))
        
fine_tune(True,current_frame=current_frame,current_frame_gray=current_frame_gray)
fine_tune(False,current_frame=current_frame,current_frame_gray=current_frame_gray)
fine_tune_res = fine_tune(current_frame=current_frame,current_frame_gray=current_frame_gray,abslut=[100,0.3,10,15])


# Part-2: Lukas-Kanade for flow extraction

In [ ]:
video_address = "./02.mp4"
video = cv2.VideoCapture(video_address)
I = 0
Frames = []
Features = []
FramesGray = []

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('./P2-Output.mp4', fourcc, 30.0, (width, height))

while True:
    I += 1
    ret, current_frame = video.read()
    current_frame = cv2.rotate(current_frame,cv2.ROTATE_180)
    if not ret :
        break
    
    gray = cv2.cvtColor(current_frame,cv2.COLOR_BGR2GRAY)
    features = cv2.goodFeaturesToTrack(gray,maxCorners=100,qualityLevel=0.3,minDistance=10,mask=None,blockSize=15,useHarrisDetector=False)
    
    Features.append(features)
    Frames.append(current_frame)
    FramesGray.append(gray)
    
    if len(Frames) > 3 :
        del Frames[0]
        del FramesGray[0]
        del Features[0]
    
    if len(Frames) < 3:
        continue
    
    points, stat, err = cv2.calcOpticalFlowPyrLK(FramesGray[1], FramesGray[2], Features[1], None,winSize  = (20, 20),maxLevel = 25,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    old_points = Features[1][stat==1]
    new_points = points[stat==1]
    frame = Frames[2].copy()
    
    for old,new in zip(old_points,new_points):
        if old is not None and new is not None:
            old = old.astype(int)
            new = new.astype(int)
            #frame = cv2.arrowedLine(frame, old, new, (250, 250, 22), 3,line_type=1,tipLength=0.4)
            frame = cv2.circle(frame,old,2,(0,255,255),3)
            frame = cv2.line(frame, old, new, (250, 250, 22), 3)#,line_type=1,tipLength=0.4)
            frame = cv2.circle(frame,new,2,(0,255,255),3)
    
    points, stat, err = cv2.calcOpticalFlowPyrLK(FramesGray[0], FramesGray[1], Features[0], None,winSize  = (20, 20),maxLevel = 25,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    old_points = Features[0][stat==1]
    new_points = points[stat==1]
    #frame = Frames[2].copy()
    
    for old,new in zip(old_points,new_points):
        if old is not None and new is not None:
            old = old.astype(int)
            new = new.astype(int)
            frame = cv2.circle(frame,old,2,(0,0,255),3)
            frame = cv2.line(frame, old, new, (250, 250, 22), 3)
            frame = cv2.circle(frame,new,2,(0,0,255),3)
            
                    
    out.write(frame)
    
    if I%30 == 0:
        cv2.imwrite("./P2-during-outputs-{0}.jpg".format(I),frame)
   
    
out.release()
    
    

# Part-3: gunner-farneback for flow extraction

In [ ]:
video_address = "./02.mp4"
video = cv2.VideoCapture(video_address)
I = 0
Frames = []
Features = []
FramesGray = []

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('./P3-Output.mp4', fourcc, 30.0, (width, height))

hsv_mask = np.zeros((height,width,3),dtype=np.uint8)
hsv_mask[:,:, 1] = 255

#vid = cv2.VideoCapture(0)
while True:
    I += 1
    """
    ret, current_frame = vid.read()
    if I == 901:
        break
    """
    ret, current_frame = video.read()
    current_frame = cv2.rotate(current_frame,cv2.ROTATE_180)
    if not ret :
        break
    
    gray = cv2.cvtColor(current_frame,cv2.COLOR_BGR2GRAY)
    features = cv2.goodFeaturesToTrack(gray,maxCorners=100,qualityLevel=0.3,minDistance=10,mask=None,blockSize=15,useHarrisDetector=False)
    
    Features.append(features)
    Frames.append(current_frame)
    FramesGray.append(gray)
    
    if len(Frames) > 3 :
        del Frames[0]
        del FramesGray[0]
        del Features[0]
    
    if len(Frames) < 3:
        continue
    
    flow = cv2.calcOpticalFlowFarneback(FramesGray[0], FramesGray[2], None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv_mask[..., 0] = ang * 180 / np.pi / 2
    hsv_mask[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb_representation = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
 
    out.write(rgb_representation)
    
out.release()
    
    